In [27]:
import json
import sys

In [28]:
time_point = ["1_9_11h","2_28h", "3_48h", "4_globular", "5_heart", "6_torpedo", "7_bent", "8_cotyledon", "9_late_cotyledon"]
time_point_n = len(time_point)
time_point_id = {}
for i in range(0,time_point_n):
    time_point_id[time_point[i]] = i

In [29]:
### read edge info 
edge = {}; node_all = []; node_each = {}; main_edge = set()

file = open("endo_edge.txt")
for line in file:
    l = line.rstrip().split('\t')

    if l[0] not in node_all:
        node_all.append(l[0])
    if l[1] not in node_all:
        node_all.append(l[1])

    edge[l[0]] = edge.get(l[0], [])
    edge[l[0]].append(l[1])

    num = time_point_id[l[0].split(':')[0]]
    node_each[num] = node_each.get(num, [])
    if l[0] not in node_each[num]:
        node_each[num].append(l[0])

    main_edge.add((l[0],l[1]))


In [18]:
### read edge weight file

main_weight = {}; extra_weight = {}

file = open("edge_prob.txt")


for line in file:

    l = line.rstrip().split('\t')

    if float(l[2])>0.8:
        xx = "specific"
    elif float(l[2])>0.2:
        xx = "additional"

    if (l[0],l[1]) in main_edge:

        if float(l[2]) > 0.2:
            main_weight[l[1]] = float(l[2])
        else:

            main_weight[l[1]] = 0
    else:
        if float(l[2]) > 0.2:
            extra_weight[(l[0],l[1])] = float(l[2])


In [36]:
### read group information, used for the color of the node
file = open("endo_celltype.list")
i = 1
coor = {}
node_group = {}
for line in file:
    l = line.rstrip().split("\t")
    node_group[l[0]] = int(l[1])
    coor[l[0]] = i
    i += 1
file.close()

In [37]:
### create the info for all the node

dat = {}

for i in node_all:

    dat[i] = {'name':i}

    if i in edge:
        dat[i]['children'] = []

    if i in main_weight:
        dat[i]['edge_weight'] = main_weight[i]
    else:
        print(i)

    if i.split(':')[1] in coor:
        dat[i]['fx'] = str(coor[i.split(':')[1]])
    else:

        print(i)

    color_map = {
        0: "#ef6547",   # 
        1: "#FF0000",   # 
        2: "#FF8C00",   # 
        3: "#C2A289",   # 
        4: "#A46D2C",   # 
        5: "#9A9AF8",   # 
        6: "#dbdcde",   # 
        7: "#A4A5A7",   # 
        8: "#FAD1E0",   # 
        9: "#F683D8",   # 
        10: "#EF5D8C",  # 
        11: "#65C2A4",  # 
        12: "#1E803D",  # 
        13: "#9FB2DA",  # 
        14: "#1B79AF",  # 
        15: "#c2dcbf",  # 
        16: "#FBE5B8",  # 
    }

    if i.split(':')[1] in node_group:
        dat[i]['node_group'] = color_map[node_group[i.split(':')[1]]]
    else:
        print(i)


1_9_11h:CC
1_9_11h:Syn


In [38]:
print("Extra weight dictionary:", extra_weight)

Extra weight dictionary: {}


In [40]:

for node in dat.values():
    node['children'] = []


for i in range(time_point_n-2, -1, -1): 
    if i in node_each: 
        for j in node_each[i]:
            if j in edge:  
                for k in edge[j]:
                    if k in dat:  
                        dat[j]['children'].append(dat[k])


root_node = {'name': 'All', 'children': []}


earliest_time_point = time_point[0]
for node in node_each[time_point_id[earliest_time_point]]:
    root_node['children'].append(dat[node])

def add_children(node):
    if 'children' in node:
        new_children = []
        for child in node['children']:
            if child['name'] in edge:
                child['children'] = [dat[child_name] for child_name in edge[child['name']] if child_name in dat]
                add_children(child)
            new_children.append(child)
        node['children'] = new_children

add_children(root_node)

def remove_empty_children(node):
    if 'children' in node:
        if not node['children']:
            del node['children']
        else:
            for child in node['children']:
                remove_empty_children(child)

remove_empty_children(root_node)

with open("endo_0.5_0529v1.json", 'w') as json_file:
    json.dump(root_node, json_file)

In [41]:
with open('endo.json', 'r') as file:
    tree_data = json.load(file)


def reorder_tree_data(node, depth=0):
    if depth < len(node_order) and 'children' in node:

        current_order = node_order[depth]

        order_map = {name: index for index, name in enumerate(current_order)}

        node['children'].sort(key=lambda x: order_map.get(x['name'], float('inf')))

        for child in node['children']:
            reorder_tree_data(child, depth + 1)

reorder_tree_data(tree_data)

def remove_empty_children(node):
    if 'children' in node:
        if not node['children']:
            del node['children']
        else:
            for child in node['children']:
                remove_empty_children(child)

remove_empty_children(tree_data)

with open('endo_0.5_reordered.json', 'w') as file:
    json.dump(tree_data, file, ensure_ascii=False)
